# Importing Dependencies

In [2]:
import os
import json
import time

from dotenv import load_dotenv
from google import genai
from google.genai import types

from typing import List

In [6]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [8]:
client = genai.Client(api_key=gemini_api_key)

# Root Features

In [9]:
# # All root features
# with open("../data/root_features.json", "r", encoding="utf-8") as file:
#     root_features = json.load(file)

# Selected root features
with open("../data/selected_features.json", "r", encoding="utf-8") as file:
    root_features = json.load(file)

# Functions

In [10]:
system_prompt = """"
"You are an expert in mobile app development and requirements engineering. 
You excel at decomposing high-level features into detailed sub-features.
"""

In [27]:
def get_level1_prompt(feature: str, feature_description: str, num_features: int = 5, technique: str = "base", examples: List[dict] = None) -> str:

    base_prompt = f"""
        **Feature**
        ```
        {feature}: {feature_description}
        ```
        Given the mobile app feature above, please refine it to a list of sub-features.
        Ensure that the number of sub-features is {num_features}.
        The output should be a list of JSON formatted objects like this:
        [{{
        "sub_feature": sub_feature,
        "description": description
        }}]
        """
    
    if technique == "base":
        return base_prompt

    elif technique == "chain_of_thought":
        return base_prompt + "\nLet's break this down step by step to identify sub-features. Think carefully before listing them. Only return the JSON as answer, as specified."

    elif technique == "few_shot":
        examples_str = "\n".join([f"Example:\nFeature: {ex['feature']}\nSub-features: {ex['sub_features']}" for ex in examples])
        return f"""
    {examples_str}

    {base_prompt}
    """

    elif technique == "contrastive_thinking":
        return base_prompt + "\nPropose two contrasting interpretations of the feature, then derive sub-features for each interpretation. Only return the JSON as answer, as specified."

    elif technique == "prompt_maieutica":
        return base_prompt + "\nLet's use a Socratic approach: What is the primary goal of this feature? What are the essential components? How do these components interact? Only return the JSON as answer, as specified."

    else:
        raise ValueError("Invalid prompt technique")



In [28]:
def get_level2_prompt(feature: str, feature_description: str, super_feature: str, super_feature_description: str,
                      siblings_features: List[str], num_features: int = 5, technique: str = "base", examples: List[dict] = None) -> str:
    base_prompt = f"""
**Super Feature**
```
super-feature: {super_feature}
description: {super_feature_description}
```
Knowing that the feature "{super_feature}" above is refined into a list of the following features:
```
{siblings_features}
```

Please refine the following to a list of sub-features.
Ensure that the number of sub-features is {num_features}.

**Feature**
```
feature: {feature}
description: {feature_description}
```

The output should be a list of JSON formatted objects like this:
[
    {{
        "sub_feature": sub_feature,
        "description": description
    }}
]
"""

    if technique == "base":
        return base_prompt

    elif technique == "chain_of_thought":
        return base_prompt + "\nLet's break this down step by step to identify sub-features. Think carefully before listing them. Only return the JSON as answer, as specified."

    elif technique == "few_shot":
        examples_str = "\n".join([f"Example:\nSuper Feature: {ex['super_feature']}\nSuper Feature Description: {ex['super_feature_description']}\nFeature: {ex['feature']}\nFeature Description: {ex['feature_description']}\nSub-features: {ex['sub_features']}" for ex in examples])
        return f"""
    {examples_str}

    {base_prompt}
    """

    elif technique == "contrastive_thinking":
        return base_prompt + "\nPropose two contrasting interpretations of the feature, then derive sub-features for each interpretation. Only return the JSON as answer, as specified."

    elif technique == "prompt_maieutica":
        return base_prompt + "\nLet's use a Socratic approach: What is the primary goal of this feature? What are the essential components? How do these components interact? Only return the JSON as answer, as specified."

    else:
        raise ValueError("Invalid prompt technique")


In [ ]:
def load_examples_from_json(directory: str, filename: str, level: int) -> List[dict]:
    file_path = os.path.join(directory, filename)

    try:
        with open(file_path, "r", encoding="utf-8") as file:
            examples = json.load(file)

        if level == 1:
            formatted_examples = [
                {
                    "feature": example["feature"],
                    "sub_features": json.dumps(example["sub_features"], indent=2)
                }
                for example in examples
            ]
        
        elif level == 2:
            formatted_examples = [
                {
                    "super_feature": example["super_feature"],
                    "super_feature_description": example["super_feature_description"],
                    "siblings_features": example["siblings_features"],
                    "feature": example["feature"],
                    "feature_description": example["feature_description"],
                    "sub_features": json.dumps(example["sub_features"], indent=2)
                }
                for example in examples
            ]
        
        else:
            raise ValueError("O parâmetro 'level' deve ser 1 ou 2.")

        return formatted_examples

    except FileNotFoundError:
        print(f"Erro: Arquivo '{filename}' não encontrado no diretório '{directory}'.")
        return []
    except json.JSONDecodeError:
        print(f"Erro: Não foi possível decodificar o JSON no arquivo '{filename}'.")
        return []

In [30]:
prompt_techniques = ["base", "chain_of_thought", "few_shot", "contrastive_thinking", "prompt_maieutica"]

# Level 1 Feature Refinements

In [ ]:
techniques_first_level_results = {}
for technique in prompt_techniques:
        
    first_level_results = []

    for feature in root_features:
        prompt = get_level1_prompt(
            feature["feature"],
            feature["description"],
            technique=technique,
            examples=load_examples_from_json("../data/examples", "few_shot_examples_level_1.json", 1) if technique == "few_shot" else None
        )

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(system_instruction=system_prompt),
            contents=prompt
        )
        
        first_level_results.append({
            "feature": feature["feature"],
            "description": feature["description"],
            "sub_features": json.loads((response.text).replace('```', '').replace('json', ''))
        })

        # Minimizar problemas de limite de requisicao por minuto
        time.sleep(5)

    techniques_first_level_results[technique] = first_level_results

    # Salvar os resultados em um arquivo JSON
    with open(f"../data/gemini/{technique}/first_level_subfeatures.json", "w") as f:
        json.dump(first_level_results, f, indent=4)

# Level 2 Feature Refinements

In [ ]:
for technique in prompt_techniques:
    second_level_results = []
    first_level_results = techniques_first_level_results[technique]

    for super_feature in first_level_results:
        print(super_feature)
        for feature in super_feature["sub_features"]:
            print(feature)
            prompt = get_level2_prompt(
                feature["sub_feature"], 
                feature["description"],
                super_feature["feature"],
                super_feature["description"],
                siblings_features=[f["sub_feature"] for f in super_feature["sub_features"]],
                technique=technique,
                examples=load_examples_from_json("../data/examples", "few_shot_examples_level_2.json", 2) if technique == "few_shot" else None
            )

            response = client.models.generate_content(
                model="gemini-2.0-flash",
                config=types.GenerateContentConfig(system_instruction=system_prompt),
                contents=prompt
            )

            second_level_results.append({
                "super_feature": super_feature["feature"],
                "super_feature_description": super_feature["description"],
                "feature": feature["sub_feature"],
                "description": feature["description"],
                "siblings": [f["sub_feature"] for f in super_feature["sub_features"]],
                "sub_features": json.loads((response.text).replace('```', '').replace('json', ''))
            })

            # Minimizar problemas de limite de requisicao por minuto
            time.sleep(5)

    # Salvar os resultados em um arquivo JSON
    with open(f"../data/gemini/{technique}/second_level_subfeatures.json", "w") as f:
        json.dump(second_level_results, f, indent=4)

[{'feature': 'Travel Planner', 'sub_features': '[\n  {\n    "sub_feature": "Flight Booking",\n    "description": "Allow users to search, filter, and book flights from various airlines, specifying dates, destinations, number of passengers, and cabin class. Integrate with flight APIs to display real-time availability and pricing. Include options for saving preferred airlines and travel preferences."\n  },\n  {\n    "sub_feature": "Personalized Itinerary Creation",\n    "description": "Enable users to create and customize daily itineraries based on their interests, travel style, and duration of the trip. Suggest attractions, restaurants, and activities based on user preferences and location. Allow manual addition of custom activities and time slots. Offer options to optimize itinerary based on travel time and opening hours."\n  },\n  {\n    "sub_feature": "Restaurant and Activity Recommendations",\n    "description": "Provide curated recommendations for restaurants, attractions, and activ

TypeError: string indices must be integers, not 'str'